In [18]:
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
import pandas as pd
import re

# 다른 경로로 csv 파일 저장
PATH = 'C:\\Education\\teamproject\\final_prioject\\news'


def crawler(company_code, start_date, end_date):
    page = 1
    
    while True:
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page)
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")
        
        titles = html.select('.title')
        title_result = [title.get_text(strip=True) for title in titles]
        
        links = html.select('.title')
        link_result = ['https://finance.naver.com' + link.find('a')['href'] for link in links]
        
        dates = html.select('.date')
        date_result = [date.get_text(strip=True) for date in dates]
        
        # 문자열 형식의 날짜를 datetime으로 변환
        converted_dates = []
        for date_str in date_result:
            try:
                # dateutil.parser.parse를 사용하여 날짜를 자동으로 파싱
                converted_date = parser.parse(date_str).date()
                converted_dates.append(converted_date)
            except ValueError:
                print(f"Ignoring invalid date: {date_str}")
        
        # 시작 날짜부터 1년 6개월 전의 뉴스만 가져오도록 필터링
        result = {"날짜": converted_dates, "기사제목": title_result, "링크": link_result}
        df_result = pd.DataFrame(result)
        df_result = df_result[(df_result['날짜'] >= start_date) & (df_result['날짜'] <= end_date)]
        
        if not df_result.empty:
            print("다운 받고 있습니다------")
            df_result.to_csv(f'{PATH}\\page{page}.csv', mode='w', encoding='utf-8-sig', index=False)
            page += 1
        else:
            print("더 이상 뉴스가 없습니다.")
            break

def convert_to_code(company, start_date, end_date):
    # 가정한 데이터프레임 생성
    data1 = pd.read_csv(f'{PATH}\\stocklistcode.csv')
    
    company_name_column = data1['종목명']
    company_code_column = data1['종목코드']
    
    keys = company_name_column.tolist()
    values = company_code_column.tolist()
    
    dict_result = dict(zip(keys, values))
    
    pattern = '[a-zA-Z가-힣]+' 
    
    if bool(re.match(pattern, company)) == True:
        company_code = dict_result.get(str(company))
        crawler(company_code, start_date, end_date)
    else:
        company_code = str(company)
        crawler(company_code, start_date, end_date)

def main():
    info_main = input("="*50+"\n"+"실시간 뉴스기사 다운받기."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    company = input("종목명이나 코드 입력: ") 
    end_date = input("종료 날짜(YYYY.MM.DD 또는 YYYYMMDD) 입력: ")
    
    try:
        end_date = datetime.strptime(end_date.replace('.', ''), '%Y%m%d').date()
    except ValueError:
        print("잘못된 날짜 형식입니다. 'YYYY.MM.DD' 또는 'YYYYMMDD' 형식으로 다시 입력해주세요.")
        return
    
    # 현재 날짜로부터 1년 6개월 전의 날짜 계산
    start_date = datetime.now() - timedelta(days=int(1.8*365/2))
    
    # 디렉토리가 없으면 생성
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    print(start_date.date(), end_date)
    convert_to_code(company, start_date.date(), end_date)

if __name__ == "__main__":
    main()


2023-03-07 2024-01-20
다운 받고 있습니다------
더 이상 뉴스가 없습니다.
